## TensorFlow 2 Complete Project Workflow in Amazon SageMaker
### Model Training
    
1. [SKLearn Linear Regression](#LogitRegression)
2. [SageMaker Linear Learner](#LinearLearner)
3. [SageMaker XGBoost](#XGBoost)
4. [Local Mode training](#LocalModeTraining)
5. [SageMaker hosted training](#SageMakerHostedTraining)
6. [Automatic model tuning](#AutomaticModelTuning)

First, we'll import the variables stored from previous notebooks.

In [ ]:
%store -r

##  SKLearn Linear Regression <a class="anchor" id="LogitRegression">

To start out with, we're going to train a scikit-learn logistic regression model like you'd normally do on your computer.

In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# We'll stitch the transformed dataset back together
# since KFold takes care of splitting for us
x_1 = np.load('./data/train/x_train.npy')
x_2 = np.load('./data/test/x_test.npy')
y_1 = np.load('./data/train/y_train.npy')
y_2 = np.load('./data/test/y_test.npy')
x = np.concatenate((x_1, x_2))
y = np.concatenate((y_1, y_2))

# Vanilla linear regression
l_regression = linear_model.LinearRegression()
kf = KFold(n_splits=10)
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scores = cross_val_score(l_regression, x, y, cv=kf, scoring='neg_mean_squared_error')
print("MSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
scores_map = {}
scores_map['LinearRegression'] = scores

# Linear regression with L2 regularization
l_ridge = linear_model.Ridge()
scores = cross_val_score(l_ridge, x, y, cv=kf, scoring='neg_mean_squared_error')
scores_map['Ridge'] = scores
print("MSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

# Polinomial regression with L2 regularization with degree for the best fit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

model = make_pipeline(PolynomialFeatures(degree=3), linear_model.Ridge())
scores = cross_val_score(model, x, y, cv=kf, scoring='neg_mean_squared_error')
scores_map['PolyRidge'] = scores
print("MSE: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

The Liner Regression with and without L2 regularization does not make significant difference is MSE score. However polynomial regression with degree=3 has a better MSE.

##  SageMaker Linear Learner <a class="anchor" id="LinearLearner">

Next we'll try SageMaker Linear Learner. The SageMaker Linear Learner algorithm provides a significant increase in speed over naive hyperparameter optimization techniques.

First we convert our numpy data into `RecordSet` objects. Although Linear Learner training supports CSV, most Amazon SageMaker algorithms work best when you use the optimized protobuf recordIO data format for training. Using this format allows you to take advantage of Pipe mode. In Pipe mode, your training job streams data directly from Amazon Simple Storage Service (Amazon S3). Streaming can provide faster start times for training jobs and better throughput. You can find more about this format [here](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html).

In [ ]:
x_train = np.load('./data/train/x_train.npy')
y_train = np.load('./data/train/y_train.npy')
x_test = np.load('./data/test/x_test.npy')
y_test = np.load('./data/test/y_test.npy')

training_data_recordset = linear_learner.record_set(train=x_train.astype('float32'),
                                                    labels=y_train.astype('float32'),
                                                    channel='train')
testing_data_recordset = linear_learner.record_set(train=x_test.astype('float32'),
                                                   labels=y_test.astype('float32'),
                                                   channel='test')

Then we create the Linear Learner model.

In [ ]:
output_location = f's3://{bucket}/{s3_prefix}/linearlearner'

linear_learner = sagemaker.LinearLearner(role=role,
                                         train_instance_count=1,
                                         train_instance_type='ml.c4.xlarge',
                                         predictor_type='regressor',
                                         output_path=output_location,
                                         sagemaker_session=sess)

Finally, we fit the model on our data.

In [ ]:
linear_learner.fit([training_data_recordset, testing_data_recordset])
linear_model_data = linear_learner.model_data

With a test MSE of 0.1, the Linear Learner model beat our more manual exploration using scikit-learn's linear regression.

##  SageMaker XGBoost <a class="anchor" id="XGBoost">

Just for fun, let's try our hand at XGBoost. But first, we need to merge the features and labels and save the data as CSV to our local disk.

In [ ]:
import pandas as pd

x_train = np.load('./data/train/x_train.npy')
x_test = np.load('./data/test/x_test.npy')
y_train = np.load('./data/train/y_train.npy')
y_test = np.load('./data/test/y_test.npy')

train_df = pd.DataFrame(data=x_train)
train_df['target'] = y_train
first_col = train_df.pop('target')
train_df.insert(0, 'target', first_col)

test_df = pd.DataFrame(data=x_test)
test_df['target'] = y_test
first_col = test_df.pop('target')
test_df.insert(0, 'target', first_col)

train_df.to_csv('./data/train/train.csv', header=False, index=False)
test_df.to_csv('./data/test/validation.csv', header=False, index=False)

We then upload those CSVs to S3.

In [ ]:
from sagemaker.session import s3_input

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(s3_prefix, 'data/train/train.csv')).upload_file('./data/train/train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(s3_prefix, 'data/validation/validation.csv')).upload_file('./data/test/validation.csv')
s3_input_train = f's3://{bucket}/{s3_prefix}/data/train/train.csv'
s3_input_test = f's3://{bucket}/{s3_prefix}/data/validation/validation.csv'
s3_input_train = s3_input(s3_input_train, content_type='csv')
s3_input_test = s3_input(s3_input_test, content_type='csv')

Now we create the XGBoost model and pass in the required hyperparameter. Details of hyperparamters can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).

In [ ]:
sess = sagemaker.Session()
container = get_image_uri(sess.boto_region_name, 'xgboost', repo_version='1.0-1')
output_location = f's3://{bucket}/{s3_prefix}/xgboost'

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=sess)

xgb.set_hyperparameters(num_round=25)

Finally, we fit the XGBoost model.

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_test})

A test RMSE of 0.19. Not bad but not as good as the Linear Learner.

##  Experiment tracking <a class="anchor" id="Experiment">
SageMaker experiments can track all the model training iterations. Experiments are a great way to organize your data science work. You can create experiments to organize all your model development work for : [1] a business use case you are addressing (e.g. create experiment named “customer churn prediction”), or [2] a data science team that owns the experiment (e.g. create experiment named “marketing analytics experiment”), or [3] a specific data science and ML project. Think of it as a “folder” for organizing your “files”.

In [ ]:
import sys

!{sys.executable} -m pip install sagemaker-experiments

In [ ]:
from smexperiments.experiment import Experiment
from time import gmtime, strftime
import boto3

sm = boto3.client('sagemaker')

experiment = Experiment.create(
    experiment_name="boston-housing-regression-{}".format(strftime("%d-%H-%M-%S", gmtime())), 
    description="Boston housing price estimation.", 
    sagemaker_boto_client=sm)

experiment_config={
    "ExperimentName": experiment.experiment_name,
    "TrialComponentDisplayName": "Training",
}

##  Local Mode training<a class="anchor" id="LocalModeTraining">
    
Before we begin, we'll pull the original Boston Housing dataset as the TensorFlow example we're about to run uses all features in its architecture.

In [ ]:
from tensodelete_endpoint.python.keras.datasets import boston_housing
import os

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# Clean up local train and test folders
filelist = [ f for f in os.listdir('./data/train') if (f.endswith(".npy") or f.endswith(".csv"))]
for f in filelist:
    os.remove(os.path.join('./data/train', f))
    
filelist = [ f for f in os.listdir('./data/test') if (f.endswith(".npy") or f.endswith(".csv"))]
for f in filelist:
    os.remove(os.path.join('./data/test', f))

# Save the original Boston Housing dataset
np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)

output_destination = 's3://{}/{}/data'.format(bucket, s3_prefix)
x_train_in_s3 = '{}/train/x_train.npy'.format(output_destination)
x_test_in_s3 = '{}/test/x_test.npy'.format(output_destination)
y_train_in_s3 = '{}/train/y_train.npy'.format(output_destination)
y_test_in_s3 = '{}/test/y_test.npy'.format(output_destination)
!aws s3 cp ./data/train/x_train.npy {x_train_in_s3}
!aws s3 cp ./data/test/x_test.npy {x_test_in_s3}
!aws s3 cp ./data/train/y_train.npy {y_train_in_s3}
!aws s3 cp ./data/test/y_test.npy {y_test_in_s3}

Local Mode in Amazon SageMaker is a convenient way to make sure your code is working locally as expected before moving on to full scale, hosted training in a separate, more powerful SageMaker-managed cluster. To train in Local Mode, it is necessary to have docker-compose or nvidia-docker-compose (for GPU instances) installed. Running the following commands will install docker-compose or nvidia-docker-compose, and configure the notebook environment for you.

In [ ]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

Next, we'll set up a TensorFlow Estimator for Local Mode training. Key parameters for the Estimator include:

- `train_instance_type`: the kind of hardware on which training will run. In the case of Local Mode, we simply set this parameter to `local` to invoke Local Mode training on the CPU, or to `local_gpu` if the instance has a GPU. 
- `git_config`:  to make sure training scripts are source controlled for coordinated, shared use by a team, the Estimator can pull in the code from a Git repository rather than local directories.  
- Other parameters of note: the algorithm’s hyperparameters, which are passed in as a dictionary, and a Boolean parameter indicating that we are using Script Mode. 

Recall that we are using Local Mode here mainly to make sure our code is working. Accordingly, instead of performing a full cycle of training with many epochs (passes over the full dataset), we'll train only for a small number of epochs just to confirm the code is working properly and avoid wasting full-scale training time unnecessarily.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.Session()
git_config = {'repo': 'https://github.com/aws-samples/amazon-sagemaker-script-mode', 
              'branch': 'master'}

model_dir = '/opt/ml/model'
train_instance_type = 'local'
hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01}
role = sagemaker.get_execution_role()

local_estimator_parameters = {'git_config': git_config,
                             'source_dir': 'tf-2-workflow/train_model',
                             'entry_point':'train.py',
                             'model_dir': model_dir,
                             'train_instance_type' : train_instance_type,
                             'train_instance_count': 1,
                             'hyperparameters': hyperparameters,
                             'role' : role,
                             'base_job_name':'tf-2-workflow',
                             'framework_version':'2.1',
                             'py_version':'py3',
                             'script_mode':True}

local_estimator = TensorFlow(**local_estimator_parameters)

The `fit` method call below starts the Local Mode training job.  Metrics for training will be logged below the code, inside the notebook cell.  You should observe the validation loss decrease substantially over the five epochs, with no training errors, which is a good indication that our training code is working as expected.

In [ ]:
inputs = {'train': f'file://{train_dir}',
          'test': f'file://{test_dir}'}

local_estimator.fit(inputs)

local_model_data = local_estimator.model_data

##  SageMaker hosted training <a class="anchor" id="SageMakerHostedTraining">

Now that we've confirmed our code is working locally, we can move on to use SageMaker's hosted training functionality. Hosted training is preferred for doing actual training, especially large-scale, distributed training.  Unlike Local Mode training, for hosted training the actual training itself occurs not on the notebook instance, but on a separate cluster of machines managed by SageMaker.  Before starting hosted training, the data must be in S3, or an EFS or FSx for Lustre file system. We'll upload to S3 now, and confirm the upload was successful.

In [ ]:
traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}

We're now ready to set up an Estimator object for hosted training. It is similar to the Local Mode Estimator, except the `train_instance_type` has been set to a SageMaker ML instance type instead of `local` for Local Mode. Also, since we know our code is working now, we'll train for a larger number of epochs with the expectation that model training will converge to an improved, lower validation loss.

With these two changes, we simply call `fit` to start the actual hosted training.

In [ ]:
train_instance_type = 'ml.c5.xlarge'
hyperparameters = {'epochs': 30, 'batch_size': 128, 'learning_rate': 0.01}

estimator_parameters = {'git_config': git_config,
                        'source_dir': 'tf-2-workflow/train_model',
                        'entry_point':'train.py',
                        'model_dir': model_dir,
                        'train_instance_type' : train_instance_type,
                        'train_instance_count': 1,
                        'hyperparameters': hyperparameters,
                        'role' : sagemaker.get_execution_role(),
                        'base_job_name':'tf-2-workflow',
                        'framework_version':'2.1',
                        'py_version':'py3',
                        'script_mode':True}

estimator = TensorFlow(**estimator_parameters)

After starting the hosted training job with the `fit` method call below, you should observe the training converge over the longer number of epochs to a validation loss that is considerably lower than that which was achieved in the shorter Local Mode training job.  Can we do better? We'll look into a way to do so in the **Automatic Model Tuning** section below. 

In [ ]:
estimator.fit(inputs, experiment_config=experiment_config)

Now that training finished, we can use SageMaker Experiments to examine the results and see how it compares to other training jobs within the experiment. Right now this is the only job captured in Experiments, but let's take a look anyway to see what data it stores.

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=experiment.experiment_name
)

experiments_df = trial_component_analytics.dataframe()
experiments_df.columns
cols = ['TrialComponentName', 'val_loss_EVAL - Last']
experiments_df = experiments_df[cols]
experiments_df.sort_values(by='val_loss_EVAL - Last')

In [ ]:
training_job_name = estimator.latest_training_job.name

As with the Local Mode training, hosted training produces a model saved in S3 that we can retrieve.  This is an example of the modularity of SageMaker: having trained the model in SageMaker, you can now take the model out of SageMaker and run it anywhere else.  Alternatively, you can deploy the model into a production-ready environment using SageMaker's hosted endpoints functionality, as shown in the **SageMaker hosted endpoint** section below.

Retrieving the model from S3 is very easy:  the hosted training estimator you created above stores a reference to the model's location in S3.  You simply copy the model from S3 using the estimator's `model_data` property and unzip it to inspect the contents.

In [ ]:
!aws s3 cp {estimator.model_data} ./model/model.tar.gz

The unzipped archive should include the assets required by TensorFlow Serving to load the model and serve it, including a .pb file:  

In [ ]:
!tar -xvzf ./model/model.tar.gz -C ./model

## Automatic Model Tuning <a class="anchor" id="AutomaticModelTuning">

So far we have simply run one Local Mode training job and one Hosted Training job without any real attempt to tune hyperparameters to produce a better model, other than increasing the number of epochs.  Selecting the right hyperparameter values to train your model can be difficult, and typically is very time consuming if done manually. The right combination of hyperparameters is dependent on your data and algorithm; some algorithms have many different hyperparameters that can be tweaked; some are very sensitive to the hyperparameter values selected; and most have a non-linear relationship between model fit and hyperparameter values.  SageMaker Automatic Model Tuning helps automate the hyperparameter tuning process:  it runs multiple training jobs with different hyperparameter combinations to find the set with the best model performance.

We begin by specifying the hyperparameters we wish to tune, and the range of values over which to tune each one.  We also must specify an objective metric to be optimized:  in this use case, we'd like to minimize the validation loss.

In [ ]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
  'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
  'epochs': IntegerParameter(10, 50),
  'batch_size': IntegerParameter(64, 256),
}

metric_definitions = [{'Name': 'loss',
                       'Regex': ' loss: ([0-9\\.]+)'},
                     {'Name': 'val_loss',
                       'Regex': ' val_loss: ([0-9\\.]+)'}]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

Next we specify a HyperparameterTuner object that takes the above definitions as parameters.  Each tuning job must be given a budget:  a maximum number of training jobs.  A tuning job will complete after that many training jobs have been executed.  

We also can specify how much parallelism to employ, in this case five jobs, meaning that the tuning job will complete after three series of five jobs in parallel have completed.  For the default Bayesian Optimization tuning strategy used here, the tuning search is informed by the results of previous groups of training jobs, so we don't run all of the jobs in parallel, but rather divide the jobs into groups of parallel jobs.  There is a trade-off: using more parallel jobs will finish tuning sooner, but likely will sacrifice tuning search accuracy. 

Now we can launch a hyperparameter tuning job by calling the `fit` method of the HyperparameterTuner object.  The tuning job may take around 10 minutes to finish.  While you're waiting, the status of the tuning job, including metadata and results for invidual training jobs within the tuning job, can be checked in the SageMaker console in the **Hyperparameter tuning jobs** panel.  

In [ ]:
from time import gmtime, strftime

tuner_parameters = {'estimator':estimator,
                    'objective_metric_name':objective_metric_name,
                    'hyperparameter_ranges':hyperparameter_ranges,
                    'metric_definitions':metric_definitions,
                    'max_jobs':4,
                    'max_parallel_jobs':2,
                    'objective_type':objective_type}

tuner = HyperparameterTuner(**tuner_parameters)

tuning_job_name = "tf-2-workflow-{}".format(strftime("%d-%H-%M-%S", gmtime()))
tuner.fit(inputs, job_name=tuning_job_name)
tuner.wait()

After the tuning job is finished, we can use the `HyperparameterTuningJobAnalytics` object from the SageMaker Python SDK to list the top 5 tuning jobs with the best performance. Although the results vary from tuning job to tuning job, the best validation loss from the tuning job (under the FinalObjectiveValue column) likely will be substantially lower than the validation loss from the hosted training job above, where we did not perform any tuning other than manually increasing the number of epochs once.  

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

The total training time and training jobs status can be checked with the following lines of code. Because automatic early stopping is by default off, all the training jobs should be completed normally.  For an example of a more in-depth analysis of a tuning job, see the SageMaker official sample [HPO_Analyze_TuningJob_Results.ipynb](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) notebook.

In [ ]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

We'll use the training artifacts created in this notebook in downstream notebooks for model deployment. Store them here for later retrieval.

In [ ]:
%store local_model_data
%store role
%store training_job_name
%store estimator_parameters
%store tuning_job_name
%store inputs

# Remove estimator key as it's not serializable.
if 'estimator' in tuner_parameters: tuner_parameters.__delitem__('estimator')
%store tuner_parameters